In [1]:
import json
from datetime import date
from pathlib import Path
from google import genai
from google.genai import types
from tickers_list import tickers_list

In [2]:
credentials_path = Path("../credentials/credentials.json")

with credentials_path.open("r", encoding="utf-8") as file:
    credentials = json.load(file)

GEMINI_API_KEY      = credentials["gemini_api_key"]
GOOGLE_CHAT_API_KEY = credentials["google_chat_api_key"]

In [3]:
prompt_path = Path("../prompts/stocks_prompt.md")

with prompt_path.open("r", encoding="utf-8") as file:
    stocks_prompt = file.read()

In [4]:
today_str   = date.today().strftime("%Y_%m_%d")
results_dir = Path("..") / "results" / today_str

results_dir.mkdir(parents=True, exist_ok=True)

In [5]:
client = genai.Client(
    api_key = GEMINI_API_KEY
)

config = types.GenerateContentConfig(
    tools = [
        types.Tool(
            google_search = types.GoogleSearch()
        )
    ]
)

In [6]:
for ticker_code in tickers_list:
    response = client.models.generate_content(
        model    = "gemini-2.5-pro",
        contents = stocks_prompt.format(ticker_code=ticker_code),
        config   = config,
    )
    
    output_file = results_dir / f"{ticker_code}_{today_str}.md"

    with output_file.open("w", encoding="utf-8") as f:
        f.write(response.text)